In [ ]:
## 5. Data Understanding

### 5.1 Data Summary

In [2]:
#Set pandas options
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
#Check the features datatypes and change it as needed
df = pd.read_csv('dados_iptu.csv')

#DataFrame informations
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40816178 entries, 0 to 40816177
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   contribuinte_n      object 
 1   CEP                 object 
 2   terreno_valorm2     float64
 3   construcao_valorm2  float64
 4   imovel_idade        float64
 5   propriedade_tipo    object 
 6   iptu_ano            int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 2.1+ GB


In [4]:
#Define function to change datatypes and speed up the following processes
def transform_dtype(data):
     #Readequar colunas com seus respectivos tipos de dados para aumento de performance -> 90% de diminuição no tamanho do dataframe
        data['imovel_idade'] = data['imovel_idade'].apply(pd.to_numeric).astype('int16')
        data['iptu_ano'] = data['iptu_ano'].apply(pd.to_numeric).astype('int16')
        data['terreno_valorm2'] = data['terreno_valorm2'].apply(pd.to_numeric).astype('float32')
        data['construcao_valorm2'] = data['construcao_valorm2'].apply(pd.to_numeric).astype('float32')
        data['propriedade_tipo'] = data['propriedade_tipo'].astype('category')
        data["CEP"] = data["CEP"].astype('category')
        data["contribuinte_n"] = data["contribuinte_n"].astype('category')

        return data

df = transform_dtype(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40816178 entries, 0 to 40816177
Data columns (total 7 columns):
 #   Column              Dtype   
---  ------              -----   
 0   contribuinte_n      category
 1   CEP                 category
 2   terreno_valorm2     float32 
 3   construcao_valorm2  float32 
 4   imovel_idade        int16   
 5   propriedade_tipo    category
 6   iptu_ano            int16   
dtypes: category(3), float32(2), int16(2)
memory usage: 976.4 MB


In [5]:
#How many rows, colunms
df.shape

(40816178, 7)

In [6]:
#Como são os dados
df.head()

,contribuinte_n,CEP,terreno_valorm2,construcao_valorm2,imovel_idade,propriedade_tipo,iptu_ano
0,0010030001-4,01104-001,100.41,129.07,1924,comercial horizontal - Padro B,1995
1,0010030002-2,01104-001,100.41,93.88,1944,comercial horizontal - Padro A,1995
2,0010030003-0,01104-001,100.41,93.88,1944,comercial horizontal - Padro A,1995
3,0010030004-9,01104-001,100.41,129.07,1944,comercial horizontal - Padro B,1995
4,0010030005-7,01104-001,100.41,93.88,1944,comercial horizontal - Padro A,1995


#### Executive Summary
Variáveis         | Explicação
:-----------------|:-------------:
CEP               | Código postal do imóvel
terreno_valorm2   | Valor por metro quadrado do terreno medido pela prefeitura
construcao_valorm2| Valor por metro quadrado do imóvel medido pela prefeitura
imovel_idade      | Ano de conclusão do empreendimento
propriedade_tipo  | Tipo de imóvel + Padrões de construção
iptu_ano          | Ano do recolhimento do IPTU

### 5.2 Data Exploration

#### 5.2.1 Features Numéricas

In [ ]:
#Resumo Estatístico das features numericas
df.dropna().describe()

Na média, os valores dos terrenos são mais caros do que os valores das construções, o que indica uma favorecimento a tese de que a localização é o mais importante na compra de um imóvel. O desvio padrão mais elevado no valor dos terrenos demonstra que o investidor deve ter mais cautela quanto a localização em relação a conservação do empreendimento na hora de comprar seu imóvel. Importante mencionar que em todas as colunas, exceto no iptu_ano, existem valores anomalos. Como valores anomalos, no valor por m2 tanto dos terreno como das construcoes observamos valores próximos de 0 reais. No ano de construção dos imoveis, tambem observamos valores próximos de 0 d.c.

In [ ]:
df.median(skipna=True)

Utilizando a mediana, operação estatística menos sensível a valores anomalos se observa que os valores de construções são mais relevantes do que os valores dos terrenos contrariando a conclusão com base na média. Portanto, sendo tambem sensato ao investidor observar as condições dos imóveis para estimar o valor total a ser pago pelo empreendimento. Outro ponto importante é que mais de 75% dos imóveis amostrados possuem menos de 40 anos de idade.

In [ ]:
#Variação da média nos preços de construção e valor dos terrenos ao longo do tempo
print(df.groupby('iptu_ano').median())

Em 20 anos, ocorreu um aumento em média de 2.100% nos valores dos terrenos em São Paulo. No mesmo período, os valores dos imóveis multiplicaram por 7.6x em média. A inflação acumulada desde o início do plano real foi de 524,88%. Ou seja, imóveis em sua maioria protegem contra a inflação.

### Possíveis Erros

In [ ]:
#Variável Imóvel_idade
df["imovel_idade"].sort_values() #Valores menores do que 1880 começam a ser muito questionáveis, pois a construção mais antiga de são paulo ainda em pé é de 1880. 

In [ ]:
df[df["imovel_idade"] < 1880].count() #Existem 1883708 registros com idades incorretas de imóveis -> cerca de 4.6% dos dados.....

In [ ]:
#Quantos imóveis valorizaram menos do que a inflação?

#Somar coluna terreno_valorm2 + construção_valorm2
df_imovel = df.loc[:,['terreno_valorm2','construcao_valorm2']].sum(axis=1).reset_index(drop=True)

#Adicionar a um novo dataframe
df_imovel = pd.concat([df,df_imovel], axis=1).rename(columns={0:"imovel_valorm2"}).loc[:, ['iptu_ano','imovel_valorm2']]

#Valores dos imoveis de 1995 e 2021
df_1995 = df_imovel.loc[df_imovel['iptu_ano'] == 1995, ['imovel_valorm2']]
df_2021 = df_imovel.loc[df_imovel['iptu_ano'] == 2021, ['imovel_valorm2']]

#
df_new = pd.concat([df_1995, df_2021], axis = 1, ignore_index = True)

#Valores dos imoveis de 1995 e 2021
df_1995 = df_imovel.loc[df_imovel['iptu_ano'] == 1995, ['imovel_valorm2']].reset_index(drop=True)
df_2021 = df_imovel.loc[df_imovel['iptu_ano'] == 2021, ['imovel_valorm2']].reset_index(drop=True)

#Series resultado da divisão entre os valores dos imóveis em 2021 por 1995
df_new = pd.concat([df_1995, df_2021], axis = 1)
df_new.head()

#Imóvel com menor valorização no periodo

#Adicionar uma coluna no webscrapping que identifique o imovel!

#Imovel com maior valorização no período